In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [2]:
bs,bptt = 64,70

## Testing the Music model

In [11]:
PATH='data/musichack/thesession/'

In [12]:
TEXT = data.Field(lower=True)
FILES = dict(train='wot_train', validation='wot_valid', test='wot_valid')

md = LanguageModelData.from_text_files(f'{PATH}', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

We will load our saved models in order to test for different inputs.

### Create the learner

In [8]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [9]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [16]:
learner.load_encoder('adam2_enc')

In [27]:
learner.load_encoder('adam2_enc')

In [100]:
learner.load_encoder('adam3_10_enc')

### Test

In [17]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [18]:
m=learner.model

In [20]:
def sample_model(m, s, l=250):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word.upper(), end='')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [21]:
sample_model(m, "M:4/4  ",l=250)

...K:MAJ|:(3=G,=A,=B,|=C2=C>=D=E>=D=E>=G|=A>=G=A>=B=C2=C>=D|=E>=D=C>=A=G>=E=C>=D|=E>=D=D>=C=D2(3=E=F=G|=C2=C>=D=E>=D=C>=D|=E>=D=C>=A=G>=E=D>=C|=A,>=D=D>=E=F>=G=A>=B|=C2=C>=B=C2:||:(3=G=A=B|=C2=C>=D=C>=B=A>=G|=A>=G=E>=G=A>=G=E>=G|=C2=C>=D=C>=B=A>=G|=A>=G=E>=G=A>=G=E>=G|=C2=C>=D=C>=B=A>=G|=A>=G=E>=G=A>=G=E>=G|=A>=G=A>=B=C>=B=A>=G|=E2=D2=C2:|M:4/4K:MAJ|:=C>=D|=E2=E>=D=E>=D=C>=D|=E>=D=E>=G=A2=G>=E|=C2=C>=D=E>=D=E>=G|=A>=G

In [86]:
sample_model(m, "M:4/4 k:maj |:(3^g,^a")

...K:MAJ|:=C2=C2=C2=C2|=B>=C=D>=B=C2=G2|=A>=B=C>=A=G>=F=E>=F|=G>=F=E>=F=D2=G2|=C2=C2=C2=C2|=B>=C=D>=B=C2=G2|=A>=B=C>=A=G>=F=E>=D|=C2=C2=C4:||:=E2=E2=E2=E2|=D>=E=D>=C=B2=B2|=C2=C2=C2=D>=C|=B2=G2=G4|=A2=A2=A2=A2|=G>=A=B>=C=D2=D2|=E>=F=G>=E=F>=E=D>=C|=B2=G2=G4:|M:4/4K:MAJ|:=E2=E2=E2=E2|=D>=E=D>=C=B2=B2|=C2=C2=C2=D>=C|=B2=G2=G4|=A2=A2=A2=A2|=G>=A=B>=A=G2=G2|=A>=B=C>=A=F2=A2|=G2=B2=G4:||:=E2=E2=E2=E

Here the input is a short sequence from another databases, from a Queen Song.

In [96]:
sample_model(m, "_b/2=e/2-_d/2-A/2-=B,/2-A,/2-A,,/2-=B,,,/2-A,,,/2-")

...=F/4=G/4=A/4=B/4=C'/4|=G=E=C=E=D=C|=D2=D2=E/2=F/2=G/2=A/2|=G2=E2=E=D|=C2=C2=C/2=D/2=E/2=F/2|=G2=G2=G=A|=G2=G2=C/2=D/2=E/2=F/2|=G2=G2=G=E|=F2=F2=E=D|=E2=E=D=C=B|=C2=C2|M:3/4K:MAJ|:=C>=D|=E2=E2=E>=D|=E2=E2=E>=D|=C2=C2=C>=D|=E2=E2=E>=D|=C2=C2=C>=D|=E2=E2=E>=D|=C2=C2=C2|=C4:||:=E>=F|=G2=G2=C>=D|=E2=E2=D>=E|=F2=F2=A>=F|=G2=E2=C>=D|=E2=E2=E>=D|=C2=C2=C>=D|=E2=E2=D>=C|=C4:|M:3/4K:MAJ|:=C>=D|=E2=E2=E>=D|=E2=E2=E>=D|=C